# 1. Sequential 모델을 넘어서: 케라스 함수형 API

- 지금까지 이 책에서 소개한 모든 신경망은 Sequential 모델을 사용하여 만들어졌습니다. Sequential 모델은 네트워크 입력과 출력이 하나라고 가정합니다. 이 모델은 층을 차례대로 쌓아 구성합니다.

- 하지만 일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요합니다. 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있습니다.

- 여러 경우에 다중 입력 모델, 다중 출력 모델, 그래프 구조를 띤 모델이 필요하지만 케라스의 Sequential 클래스를 사용해서는 만들지 못합니다. 케라스에는 훨씬 더 일반적이고 유연한 다른 방법인 함수형 API가 있다. 이 절에서 함수형 API가 무엇인지 소개하고 함수형 API를 사용하는 방법과 이를 할 수 있는 것을 자세히 설명하겠습니다.

# 2. 함수형 API

- 함수형 API에서는 직접 텐서들의 입출력을 다룹니다. 함수처럼 층을 사용하여 텐서를 입력받고 출력합니다.

In [1]:
from keras import Input, layers
from keras.datasets import mnist
# 함수형 API는 직접 텐서들의 입출력을 다룹니다. 함수처럼 층을 사용하여 텐서를 입력받고 출력합니다.

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')
output_tensor = dense(input_tensor)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
# My code
input_tensor = Input(shape=(64,))
dense1 = layers.Dense(32, activation='relu')
dense2 = layers.Dense(32, activation='relu')
output = layers.Dense(10, activation='softmax')

fc1 = dense1(input_tensor)
fc2 = dense2(fc1)
output_tensor = output(fc2)

model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [4]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

In [5]:
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [8]:
import numpy as np

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 0s 303us/step - loss: 11.5110
Epoch 2/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.5021
Epoch 3/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.4996
Epoch 4/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.4971
Epoch 5/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.4946
Epoch 6/10
1000/1000 [==============================] - 0s 24us/step - loss: 11.4920
Epoch 7/10
1000/1000 [==============================] - 0s 24us/step - loss: 11.4900
Epoch 8/10
1000/1000 [==============================] - 0s 24us/step - loss: 11.4884
Epoch 9/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.4867
Epoch 10/10
1000/1000 [==============================] - 0s 61us/step


In [11]:
print(score)

11.4808760223


# 2. 다중 입력 모델

- 다중 API는 다중 입력 모델을 만드는 데 사용할 수 있다. 일반적으로 이런 모델은 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용합니다. 텐서를 더하거나 이어 붙입니다. 이와 관련된 케라스 함수는 keras.layers.add, keras.layers.concatenate입니다.

### 질문 응답 모델

- 전형적인 질문 응답 모델은 2개의 입력을 가진다. 하나는 자연어 질문이고, 또 하나는 답변에 필요한 정보가 담겨 있는 텍스트입니다. 그러면 모델은 답을 출력해야 합니다. 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통해 한 단어로 된 답을 출력한다.

In [12]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

In [17]:
# 입력의 크기가 64인 벡터 시퀀스로 임베딩한다.
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# 입력의 크기가 64인 백터 시퀀스로 임베딩한다.
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 64)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# concanenate
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# answer
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# model concatenation
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 64)     640000      question[0][0]                   
__________________________________________________________________________________________________
lstm_5 (LS

In [25]:
import numpy as np 
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answer = to_categorical(answers)

In [28]:
text.shape, question.shape, answers.shape, answer.shape

((1000, 100), (1000, 100), (1000,), (1000, 500))

In [30]:
model.fit([text, question], answer, epochs=10, batch_size=128)

# dictionary input
# model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 6.2145 - acc: 1.0000e-03
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1962 - acc: 0.0410
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1444 - acc: 0.0110
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0479 - acc: 0.0100
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.9588 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8404 - acc: 0.0090
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.7599 - acc: 0.0200
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6828 - acc: 0.0320
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6090 - acc: 0.0360
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5391 - acc: 0.0530


## 3. 다중 출력 모델

- 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이, 성별, 소득 수준을 예측으로 합니다.

In [72]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

In [73]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')

# input_length를 안 정하면 conv1d가 무엇을 해야 할 지 모른다.
embedded_posts = layers.Embedding(vocabulary_size, 256, input_length=max_length)(posts_input)

# parameter 사이즈: 256 x 128 x 5 + 128
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x) # 나이
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

# model
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 500, 256)     12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 496, 128)     163968      embedding_13[0][0]               
__________________________________________________________________________________________________
max_pooling1d_13 (MaxPooling1D) (None, 99, 128)      0           conv1d_31[0][0]                  
__________________________________________________________________________________________________
conv1d_32 

In [74]:
# 각각의 층마다 손실 함수가 다르다.
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# model.compile(optimizer='rmsprop',
#              loss={'age': 'mse',
#                    'income': 'categorical_crossentropy',
#                    'gender': 'binary_crossentropy'})

- 경사 하강법은 하나의 스칼라 값을 최소화 하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 합니다. 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것입니다. 케라스에서는 compile 메서드에 리스트나 딕셔너리를 사용하여 출력마다 다른 손실을 지정할 수 있습니다.

- 손실 값이 많이 불균형하면 모델이 개별 손실이 갖아 큰 작업에 치우쳐 표현을 최적화할 것입니다. 그 결과 다른 작업은 손해를 봅니다. 이를 해결하기 위해 손실 값이 최종 손실에 기여하는 수준을 지정할 수 있습니다. 특히 손실 값의 스케일이 다를 때 유용합니다.

- 나이 회귀 작업에 사용되는 평균 제곱 오차 손실은 일반적으로 3 ~ 5 사이 값을 가집니다. 반면에 성별 분류 작업에 사용되는 크로스엔트로피 손실은 0.1 정도로 낮습니다. 이런 환경에서 손실에 균형을 맞추려면 크로스엔트로피 손실에 가중치 10을 주고 mse 손실에 가중치 0.25를 줍니다.

In [75]:
num_samples = 10000
max_length = 500

In [76]:
posts = np.random.randint(1, vocabulary_size, size=(num_samples, max_length))
ages = np.random.randint(1, 100, size=num_samples)
income = np.random.randint(1, 10, size=num_samples)
gender = np.random.randint(0, 1, size=num_samples)

In [77]:
age_targets = ages
income_targets = to_categorical(income)
gender_targets = to_categorical(gender)

In [78]:
posts.shape, age_targets.shape, income_targets.shape, gender_targets.shape

((10000, 500), (10000,), (10000, 10), (10000, 1))

In [79]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

Epoch 1/10
10000/10000 [==============================] - 6s 558us/step - loss: 291.3245 - age_loss: 1155.2206 - income_loss: 2.3870 - gender_loss: 0.0132
Epoch 2/10
10000/10000 [==============================] - 3s 342us/step - loss: 208.4401 - age_loss: 824.6087 - income_loss: 2.2878 - gender_loss: 1.8840e-05
Epoch 3/10
10000/10000 [==============================] - 3s 340us/step - loss: 111.3566 - age_loss: 436.2864 - income_loss: 2.2828 - gender_loss: 2.2562e-04
Epoch 4/10
10000/10000 [==============================] - 4s 363us/step - loss: 71.3523 - age_loss: 276.3160 - income_loss: 2.2725 - gender_loss: 7.3663e-05
Epoch 5/10
10000/10000 [==============================] - 3s 338us/step - loss: 60.6190 - age_loss: 233.3628 - income_loss: 2.2782 - gender_loss: 1.2472e-05
Epoch 6/10
10000/10000 [==============================] - 3s 340us/step - loss: 50.5735 - age_loss: 193.2026 - income_loss: 2.2728 - gender_loss: 1.6418e-06
Epoch 7/10
10000/10000 [==============================] - 

## 4. 층으로 구성된 비순환 유향 그래프

- 함수형 API를 사용하면 다중 입력이나 다중 출력뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있다. 케라스의 신경망은 층으로 구성된 어떤 비순환 유향 그래프도 만들 수 있습니다. 비순환이라는 것이 중요합니다. 다시 말해 이 그래프는 원형을 띨 수 없습니다. 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없습니다. 만들 수 있는 루프는 순환 층의 내부에 있는 것뿐입니다.

- 그래프로 구현된 몇 개 신경망 컴포넌트가 널리 사용됩니다. 가장 유명한 2개는 인셉션 모듈과 잔차 연결입니다. 케라스에서 이 2개의 컴포넌트를 어떻게 구현하는지 살펴보겠습니다.

## 1. 인셉션

In [94]:
input_tensor = Input(shape=(256, 256, 3), dtype='float32')

# first route
r1 = layers.Conv2D(64, (1, 1), strides=2, activation='relu')(input_tensor)

# second route
r2 = layers.Conv2D(32, (1, 1), strides=1, activation='relu', padding='same')(input_tensor)
r2 = layers.Conv2D(64, (3, 3), strides=2, activation='relu', padding='same')(r2)

# third route
r3 = layers.AveragePooling2D(pool_size=(3, 3), strides=2, padding='same')(input_tensor)
r3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(r3)

r4 = layers.Conv2D(32, (1, 1), activation='relu')(input_tensor)
r4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(r4)
r4 = layers.Conv2D(64, (3, 3), activation='relu', strides=2, padding='same')(r4)

output_tensor = layers.concatenate([r1, r2, r3, r4], axis=-1)

- 1x1 convolution: 채널 공간 방향으로 상관관계가 크고 채널 간에는 독립적이라고 가정하면 납득할만한 전략이다.

## 2. 잔차 연결

- 대규모 딥러닝 모델에서 흔히 나타나는 2가지 문제인 그래디언트 소실과 표현 병목을 해결했습니다.

- 하위 층의 출력이상위 층의 활성화 출력에 연결되는 것이 아니고 더해집니다. 따라서 두 출력의 크기가 동일해야 합니다.

- 잔차 연결은 하위 층의 출력을 상위 층의 입력으로 사용합니다. 순서대로 놓인 네트워크를 질러가는 연결이 만들어진다. 하위 층의 출력이 상위 층의 활성화 출력에 연결되는 것이 아니고 더해집니다.

In [103]:
from keras import layers

x = Input(shape=(224, 224, 64))
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

# concatenate이 아니라 더해진다.
y = layers.add([y, x])

In [104]:
y

<tf.Tensor 'add_4/add:0' shape=(?, 224, 224, 128) dtype=float32>

In [105]:
from keras import layers

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
y = layers.add([y, residual])

In [106]:
y

<tf.Tensor 'add_5/add:0' shape=(?, 112, 112, 128) dtype=float32>

### 표현 병목

- 딥러닝 층은 이전 층의 활성화 정보만 사용합니다. 어떤 층이 너무 작으면 이 활성화 출력에 얼마나 많은 정보를 채울 수 있느냐에 모델 성능이 좌우된다. 이 개념을 신호 처리에 비유할 수 있습니다. 일련의 연산으로 구성된 오디오 처리 파이프라인이 있다고 해보자. 각 단계는 이전 연산의 출력을 입력으로 사용합니다. 한 연산이 신호를 저주파 영역으로 잘라냈다면 이후 연산이 줄어든 주파수를 복원할 수 없을 것이다. 손실된 정보는 영구 불변이다. 하위 층의 정보를 다시 주입하는 잔차 연결은 딥러닝 모델에서 이 이슈를 어느 정도 해결합니다.

### 딥러닝의 그래디언트 소실 문제

- 피드백 신호가 깊이 쌓인 층을 통과하여 전파되면 신호가 아주 작아지거나 완전히 사라질 수 있다. 이렇게 되면 네트워크가 훈련되지 않는다. 이를 그래디 언트 소실이라 한다. 이 문제는 심층 신경망과 긴 시퀀스를 처리하는 순환 신경망에서 모두 나타납니다. 양쪽 모두 피드백 신호가 일련의 긴 연산을 통과하여 전파되기 때문입니다.

- 순환 신경망에서 LSTM 층이 이 문제를 해결하기 위해 사용하는 방식을 보았습니다. 이동 트랙이 주요 처리 트랙에 나란한 정보를 실어 날랐습니다. 잔차 연결은 피드포워드 신경망에서 비슷한 역할을 합니다. 하지만 좀 더 단순합니다. 주 네트워크 층에 나란히 단순한 선형 정보를 실어 나릅니다. 이는 그래디언트가 깊게 쌓인 층을 통과하여 전파하도록 도와줍니다.

# 5. 층 가중치 공유

- 함수형 API의 중요한 또 하나의 기능은 층 객체를 여러 번 재사용할 수 있다는 것입니다. 층 객체를 두 번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가중치를 재사용합니다. 이런 기능 때문에 공유 가지를 가진 모델을 만들 수 있습니다. 이런 가지는 같은 가중치를 공유하고 같은 연산을 수행합니다. 다시 말해 같은 표현을 공유하고 이런 표현을 다른 입력에서 함께 학습합니다.

- 예를 들어 두 문장 사이 의미가 비슷한지 측정하는 모델을 가정한다. 이 모델은 2개의 입력을 받고 0과 1 사이의 점수를 출력합니다. 0은 관련 없는 문장을 의미하고 1은 두 문장이 동일하거나 재구성되었다는 것을 의미한다. 이런 모델은 대화 시스템에서 자연어 질의에 대한 중복 제거를 포함하여 많은 어플리케이션에서 유용하게 사용될 수 있습니다.



- 이런 문제에서는 두 입력 시퀀스가 바뀔 수 있습니다. 의미가 비슷하다는 것은 대칭적인 관계이기 때문입니다. A에서 B에 대한 유사도는 B에서 A에 대한 유사도와 같습니다. 이런 이유 때문에 각 입력 문장을 처리하는 2개의 독립된 모델을 학습하는 것은 이치에 맞지 않습니다. 그 대신 LSTM 층으로 양쪽을 모두 처리하는 것이 좋습니다. 이 LSTM 층의 표현은 두 입력에 대해 학습됩니다. 이를 샴 LSTM이 합니다. 또는 공유 LSTM이라 합니다.

In [107]:
from keras import layers
from keras import Input
from keras.models import Model

# 새로운 층 객체를 만드는 것이 아니라 각 호출에 동일한 가중치를 재사용한다.
lstm = layers.LSTM(32)

# 모델의 왼쪽 가지를 구성합니다. 입력은 크기가 128인 벡터의 가변 길이 시퀀스입니다.
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 모델의 오른쪽 가지를 구성합니다. 기존 층 객체를 호출하면 가중치가 재샤용됩니다.
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)

# 맨 위에 분류기를 놓습니다.
predictions = layers.Dense(1, activation='sigmoid')(merged)

# 모델 객체를 만들고 훈련합니다. 이런 모델을 훈련하면 LSTM 가중치는 양쪽 입력을 바탕으로 업데이트됩니다.
model = Model([left_input, right_input], predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 32)           20608       input_20[0][0]                   
                                                                 input_21[0][0]                   
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 64)           0           lstm_7[0][0]                     
          

## 6. 층과 모델

- 함수형 API에서는 모델을 층처럼 사용할 수 있습니다. 모델을 '커다란 층'으로 생각해도 됩니다. Sequential 클래스와 Model 클래스에서 모두 동일합니다.

In [108]:
# y = modle(x)
# y1, y2 = model([x1, x2])

- 모델 객체를 호출할 때 모델 가중치가 재사용됩니다. 층 객체를 호출할 때와 정확히 같습니다. 층 객체나 모델 객체나 객체를 호출하는 것은 항상 그 객체가 가진 학습된 표현을 재사용합니다.

- 듀얼 카메라에서 입력을 받는 비전 모델의 경우에 왼쪽 카메라와 오른쪽 카메라에서 시각적 특징을 추출하여 합칠 떄 2개의 독립된 모델을 사용할 필요가 없습니다. 두 입력에 저수준 처리 과정이 공유될 수 있다. 다시 말해 가중치가 같고 동일한 표현을 공유하는 층을 사용합니다. 다음은 케라스에서 샴 비전 모델을 구현하는 예입니다.

In [110]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

# 7. 정리

- 차례대로 층을 쌓는 것 이상이 필요할 때는 Sequential API를 사용하지 않는다.
- 함수형 API를 사용하여 다중 입력, 다중 출력, 봊갑한 네트워크 토폴로지를 갖는 케라스 모델을 만드는 방법
- 다른 네트워크 가지에서 같은 층이나 모델 객체를 여러 번 호출하여 가중치를 재사용하는 방법